In [28]:
def build_volume_manifest(volume_id, volume_title, volume_geography, image_details, manifest_server=None, image_server=None, default_server=None, output_dir_prefix=None):
    '''
    Builds a IIIF manifest for an SSDA volume based on specified metadata.
        volume_id: unique ID for specified volume
        volume_geography: a dictionary containing the volume's location within SSDA geographic hierarchy (used to build S3 url)
        image_details: a dictionary containing the file names, extensions, and dimensions of all images in the volume
        output_dir_prefix: path to append output manifest file name to, if applicable
        manifest_server: base URL for manifest server, including trailing forward slash
        
        returns: path to a json file containing a valid IIIF manifest for the volume
    '''    
    if default_server == None:
        default_server = "http://localhost:3000/"
    
    if output_dir_prefix != None:
        volume_json_path = output_dir_prefix + '\\' + str(volume_id) + ".json"
    else:
        volume_json_path = str(volume_id) + ".json"
        
    with open(volume_json_path, "w") as outfile:
        outfile.write('{\n')
        #add indentation eventually?
        outfile.write("\"@context\": \"http://iiif.io/api/presentation/2/context.json\",\n")
        outfile.write("\"@type\": \"sc:Manifest\",\n")
        outfile.write("\"@id\": \"")
        write_server(outfile, manifest_server, default_server)           
        outfile.write("manifest/" + str(volume_id) + ".json\",\n")
        outfile.write("\"label\": \"" + volume_title + "\",\n")
        #add description
        outfile.write("\"attribution\": \"Slave Societies Digital Archive\",\n")
        outfile.write("\"logo\": \"")
        write_server(outfile, manifest_server, default_server) 
        outfile.write("ssda_logo.jpg\",\n")
        
        outfile.write("\"sequences\": [\n")
        outfile.write("{\n")
        outfile.write("\"@type\": \"sc:Sequence\",\n")
        outfile.write("\"@id\": \"")
        write_server(outfile, manifest_server, default_server)           
        outfile.write("sequence/" + str(volume_id) + ".json\",\n")
        outfile.write("\"canvases\": [\n")
        
        first_image = True
        for image in image_details:
            if not first_image:
                outfile.write(",\n")
            else:
                first_image = False
            
            outfile.write("{\n")
            outfile.write("\"@type\": \"sc:Canvas\",\n")            
            outfile.write("\"@id\": \"")
            write_server(outfile, manifest_server, default_server)
            canvas_id = str(volume_id) + '-' + str(image["file_name"])
            outfile.write("canvas/" + canvas_id + ".json\",\n")
            outfile.write("\"label\": \"" + str(image["file_name"] - 1000) + "\",\n")
            outfile.write("\"width\": " + str(image["width"]) + ",\n")
            outfile.write("\"height\": " + str(image["height"]) + ",\n")
            outfile.write("\"images\": [\n")
            outfile.write("{\n")
            outfile.write("\"@type\": \"oa:Annotation\",\n")
            #add annotation id
            outfile.write("\"motivation\": \"sc:painting\",\n")
            outfile.write("\"on\": \"")
            write_server(outfile, manifest_server, default_server)
            outfile.write("canvas/" + canvas_id + ".json\",\n")
            outfile.write("\"resource\": {\n")            
            outfile.write("\"@type\": \"dctypes:Image\",\n")
            #add resource format
            outfile.write("\"@id\": \"")
            write_server(outfile, image_server, default_server)
            outfile.write(volume_geography["country"].replace(' ', '_') + "%2F" + volume_geography["state"].replace(' ', '_') + "%2F" + volume_geography["city"].replace(' ', '_') + "%2F" + volume_geography["institution"].replace(' ', '_') + "%2F" + str(volume_id) + "%2F" + image["extension"].upper() + "%2F" + str(image["file_name"]) + '.' + image["extension"] + "/full/" + str(image["width"]) + ",/0/default.jpg\",\n")
            outfile.write("\"width\": " + str(image["width"]) + ",\n")
            outfile.write("\"height\": " + str(image["height"]) + ",\n")
            outfile.write("\"service\": {\n")
            outfile.write("\"@id\": \"")
            write_server(outfile, image_server, default_server)
            outfile.write(volume_geography["country"].replace(' ', '_') + "%2F" + volume_geography["state"].replace(' ', '_') + "%2F" + volume_geography["city"].replace(' ', '_') + "%2F" + volume_geography["institution"].replace(' ', '_') + "%2F" + str(volume_id) + "%2F" + image["extension"].upper() + "%2F" + str(image["file_name"]) + '.' + image["extension"] + "\",\n")
            outfile.write("\"@context\": \"http://iiif.io/api/image/2/context.json\",\n")
            outfile.write("\"profile\": \"http://iiif.io/api/image/2/level2.json\"\n")
            outfile.write("}\n")
            outfile.write("}\n")
            outfile.write("}\n")
            outfile.write("]\n")
            outfile.write("}")
            
        outfile.write("\n")
        outfile.write("]\n")
        outfile.write("}\n")
        outfile.write("]\n")
        outfile.write("}\n")
    
    return volume_json_path

In [20]:
def write_server(outfile, server, default):
    if server == None:
        outfile.write(default)
    else:
        outfile.write(server)

In [18]:
volume_geography = {"country": "Brazil", "state": "Minas Gerais", "city": "Nova Lima", "institution": "Nossa Senhora do Pilar"}
first_image = {"file_name": 1001, "extension": "JPG", "height": 3648, "width": 2736}
second_image = {"file_name": 1002, "extension": "JPG", "height": 3648, "width": 2736}
images = [first_image, second_image]

In [29]:
build_volume_manifest(2223, "Test Volume", volume_geography, images, manifest_server = "https://danielgenkins.com/", image_server = "http://localhost:8182/iiif/2/")

'2223.json'